
# マイクロマウスの直進モデルに対する速度制御系の設計

状態方程式は 1 階連立型線形微分方程式の形式で表されているので，数値計算をするのは比較的簡単です．  
計算法にはいくつか種類がありますが，ここではPythonの「control」というライブラリを使ってみます．

>ここでは外部のライブラリを使用するので，あらかじめ`control`と`slycot`をインストールしておいてください．   
pipが入っていれば，端末を開いて

>    sudo pip install control  
>    sudo pip install slycot
    
>とコマンドを打てば，あとは自動でインストールできます．
Macの場合，Fortranのコンパイラが必要になるかもしれませんので，予めインストールしておいてください．  
(homebrewでは，`brew install gcc`でfortranも一緒に入ります)

状態方程式モデルの応答を数値計算する際は，python-controlのStateSpaceクラスを使うと簡単です．numpyのarrayクラスを使って係数行列
$A,B,C,D(=0)$を作成し，StateSpace(A,B,C,D)のように引数を与えることで状態方程式モデルのインスタンスを作成できます．

---

## モーターに一定値入力を与えてみる

システムの特性を直感的に理解するために，まずは何も考えず一定値入力を与えてみます．  
一定値入力 $u(t)=1,t >0$ は制御工学の分野で**ステップ入力**と呼ばれます(波形が段差:stepのように見えることから)．  

このようなステップ入力を与えたときの応答を調べるといった操作はよく行われるので，python-controlにも
専用の関数(`control.matlab.step`)が用意されています．  
また，`ipywidgets`には，スライドバーのようなGUI要素を用いて，インタラクティブに数値を操作できる機能が用意されているので，
今回はこれを使って，色々パラメータを変更してみましょう．

操作できる変数
* gear_ratio: ギアの減速比(モーターギアの歯数：タイヤギアの歯数 = $1:n$)
* mass_gram: 機体の質量[g]
* motor_duty: 入力PWM信号のduty比

In [1]:
from control import *
import numpy as np
from matplotlib import pylab as plt
import matplotlib 
%matplotlib inline
plt.rcParams["font.size"] = 13
from __future__ import print_function
from ipywidgets import interact, FloatSlider, RadioButtons

# パラメータの設定
# 回路定数 (MK06-4.5の場合)
R = 5.0; # ohm
KE = 7.1504e-04; # 逆起電力定数[V/(rad/s)]
KT = 6.5e-04; # トルク定数[Nm/I]
Vbat = 4.0; # モータドライバの電源電圧[V]

# 機体パラメータ
#     Ng = 3.0;          # ギアの減速比(モーター：ホイール=1:n), スライドバーで設定
r = 7.5e-03;    # タイヤ半径[m]
#     m = 75e-03;     # 機体重量[kg], スライドバーで設定
w = 30e-03;     # 機体の中心から右端までの距離(横幅/2) [m]
h = 45e-03;     # 機体の中心から先端までの距離(縦幅/2) [m]
# J = 1/3*(w**2+h**2)*m; # 機体の慣性モーメント(形状を長方形に近似)

# スライドバーで動かせるプロットを作成
@interact(gear_ratio=(1.0, 10, 0.1), mass_gram=(5, 200, 0.5), motor_duty=(0, 1.0, 0.01))
def sim_trans_system(gear_ratio=3.0, mass_gram=30,motor_duty=0.5):
    Ng = gear_ratio; # ギアの減速比(モーター：ホイール=1:n)
    m = mass_gram/1000; # 機体の質量[kg] (スライドバーからはgで渡される)
    
    # 状態空間形式(dX/dt = AX + Bu, y = Cx)でシステムを表現    
    a = -(KT*KE*Ng**2)/(R*m*r**2)
    b = (2*motor_duty*Vbat) * (KT*Ng)/(m*r*R) # Vm = (2*motor_duty*Vbat)*u と置いた
    c = 1/(Ng*r)
    A = np.array([ [0,1,0], [0,a,0], [0,c,0]]) 
    B = np.array([[0], [b], [0]])
    C = np.array([[1,0,0],[0,1,0],[0,0,1]])
    D = np.array([[0],[0],[0]])
    sys = StateSpace(A, B, C, D)

    # 線形シミュレーション:ステップ入力 u(t)=1 (t>0)
    t = np.linspace(0, 20, 1000)
    yout, T = matlab.step(sys, t) 
    v_array = yout[:,1]
    dphi_array = v_array*c
    I_array = 2*(-KE*dphi_array+(motor_duty*Vbat))/R

    # シミュレーション結果のプロット
    fig, (axL, axR) = plt.subplots(ncols=2, figsize=(16,6))
    axL.plot(T, v_array)
    axL.axhline(b/-a, color="b", linestyle="--");
    axL.set_xlim(0, 20);  axL.set_ylim(0,30);
    axL.set_xlabel('time t[s]');axL.set_ylabel('speed vx [m/s]')
    axL.set_title('  Forward Speed (Vm: Step Input)');  # 日本語使えないつらい
        
    axR.plot(T, I_array)
    axR.set_xlim(0, 20);
    axR.set_xlabel('time t[s]');axR.set_ylabel('current I_total [A]')

    axR.set_title('Total Current of 2 Motors [A]');  # 日本語使えないのほんとウンチ
    
    plt.show();


DCモーターを理解する上で，次のポイントを抑えておくといいでしょう．

* 定常速度は電圧に比例
* ギアの減速比$N_G$を上げると，加速性能が上がるが，最高速度は下がる
* 流れる電流は始動時が一番大きい

---

それでは，このような特性を持つマウスのダイナミクスを制御することについて考えていきましょう．

ただ，このままだと，シミュレーションを書くのにコードが長くなりそうなので， 先にマウスの線形モデルを表現するためのクラスを用意しておきました．

詳細に関しては，mouse.pyを参照してください．

In [3]:
from mouse import *

# インスタンスの生成
config = MouseConfig()

config.R = 5.0; # 抵抗 [ohm]
config.KE = 7.1504e-04; # 逆起電力定数[V/(rad/s)]
config.KT = 6.5e-04; # トルク定数[Nm/I]
config.Vbat = 4.0; # モータドライバの電源電圧[V]
config.Ng = 3.0;          # ギアの減速比(モーター：ホイール=1:n), スライドバーで設定
config.r = 7.5e-03;    # タイヤ半径[m]
config.m = 50e-03;     # 機体重量[kg], スライドバーで設定
config.w = 30e-03;     # 機体の中心から右端までの距離(横幅/2) [m]
config.h = 45e-03;     # 機体の中心から先端までの距離(縦幅/2) [m]
config.J = 1/3*(config.w**2+config.h**2)*config.m; # 機体の慣性モーメント(形状を長方形に近似)
mouse = MouseDynamics(config)


それでは以上の直進走行モデルを用いて速度の制御系を立ててみます．  
まずは，第一ステップとして，ステップ入力で目標速度値が与えられたとき，速度が$t\to \infty$で偏差なく追従することを目指してみましょう．

以下に示すのは，一番ベーシックなフィードバック制御系のブロック線図です．  
Pは制御対象でプラントと呼ばれることもあります．今回の例では，入力がdutyで出力が速度なシステムだと考えてください．  
    Kはコントローラを表します．基準速度$v_{ref}$と実際の出力$v$との偏差$e$を入力し，内部で何らかの演算を行うことで制御対象$P$へ与える入力値$u$を決定します．  
$K$のとり方は自由ですが，今回はメジャーな制御法の一つであるPID制御器を設計してみましょう．

<img src="img/FeedbackSpeedControl.PNG" width=70%>


PID制御器は時間領域では，
$$
    K(t) = K_p e(t)  + K_i \int_0^t e(t)dt + K_d \dfrac{de(t)}{dt}
$$

ラプラス変換形式(伝達関数形式)では，

$$
    K(s) = K_p E(s)  + K_i \dfrac{1}{s} E(s) + K_d sE(s) \\
        = ( \dfrac{K_d s^2 + K_p s +  K_i}{s})E(s)
$$

と表すことができます．両者の式は等価なので，どちらを使っても同じ結果になりますが，
ラプラス変換の方がブロック線図と相性がよく扱いやすいのでこちらを使います．  
python-controlでは，連立微分方程式で表現された状態方程式とラプラス変換で表現された伝達関数が混在していても，
自動で計算に都合の良い形式にモデルを変換してくれます．便利ですね．

---

それでは，ステップ入力で与えられた目標速度にうまく追従するようにPIDのパラメータを調整してみましょう．
ただし，モーターへ与えることのできるdutyは最大100％であるので，これを超えないようにしましょう．
（オーバーすると，プロットが赤色になります）


In [4]:
# ステップ目標値を与えたときの，FB制御系のシミュレーション
@interact(Kp=(0.01, 3, 0.01), Ki=(0, 5, 0.001), Kd=(0, 5, 0.01), target=(0,3,0.1))
def sim_closed_loop(Kp=1.0, Ki=0, Kd=0, target=1):
    
    # PIDコントローラ（伝達関数形式）の作成
    # 純粋微分器は計算不可能なので，1次ローパスフィルタを接続した実用微分器(Kd*s/(gamma*Kd*s+1))を使う
    D_COEFFICIENT = 0.1 # 微分係数
    Kp = matlab.tf([Kp], [1])
    Ki = matlab.tf([Ki], [1, 0])
    Kd = matlab.tf([Kd, 0], [D_COEFFICIENT*Kd, 1])  
    K = Kp+Ki+Kd

    dmux = StateSpace(0,np.array([0,0,0]),0,np.array([0,1,0]) )
    P = series(mouse.ss , dmux) # 3出力あるうちの2番目(v)だけを取り出す． 

    # 閉ループ系の伝達関数を計算
    Ysys = matlab.feedback(K*P, 1, -1)  # ネガティブフィードバック
    Esys = matlab.feedback(1, K*P, -1)
    Usys = K/(1+K*P)
    
    # 線形シミュレーション
    SIM_TIME = 5    # シミュレーション時間
    SAMPLE_RATE = 100 # サンプリングレート
    t = np.linspace(0, SIM_TIME, SIM_TIME*SAMPLE_RATE)
    ref = np.ones(SIM_TIME*SAMPLE_RATE) * target # u(t)=duty (const)
    yout, T, xout = matlab.lsim(Ysys, ref, t)
#     Eout, T, xout = matlab.lsim(Esys, ref, t)
    Uout, T, xout = matlab.lsim(Usys, ref, t)
    
    
    
    if np.amax(Uout) > 1.0 :
        _is_saturated = True
    else :
        _is_saturated = False
        
    # シミュレーション結果のプロット
    fig, (axL, axR) = plt.subplots(ncols=2, figsize=(16,6))
    
    axL.plot(T, yout)
    axL.axhline(target, color="k", linestyle="--");
    axL.set_xlim(0, SIM_TIME);  axL.set_ylim(0,5);
    axL.set_xlabel('time t [s]'); axL.set_ylabel('Speed vx [m/s]')
    axL.set_title('Speed Control');  
    # axL.tick_params(labelsize=13)

    if _is_saturated: axR.plot(T, Uout, 'r',  lw=5)
    else: axR.plot(T, Uout, 'b')
    axR.set_xlim(0, SIM_TIME);
    axR.set_xlabel('time t [s]');axR.set_ylabel('duty [.]')
    axR.set_title('Average Duty Output To Two Motors');  
    # axR.tick_params(labelsize=13)

    plt.show();


目標速度$v_{target}$と$Kp,Ki,Kd$のスライダーを動かして，いろいろ試してみましょう．


---

どうだったでしょうか？  
今回の問題設定では，$t\to \infty$で漸近安定にするためにはP制御があれば十分であり，もし定常偏差も0にしたいなら$Ki$を増やせば良いことがわかります．  

筆者の設定したパラメータでは，$v_{target}=1.0 $[m/s]のとき，$Kp=0.84, Ki=0.08$で入力dutyに20%ほどの余裕を残すことができました．


![@aaa](img/control_result01.PNG)


目標速度に追従する，という点に関しては，以上の制御法で目的が達成されたと言えます．
しかし，ここはコンマ一秒を争うマイクロマウスの世界，どうせならできるだけ高速かつ高加速度を追求したいものです．

そういったことを考えると実は，先程の制御系はいくつかの問題点を抱えています．その問題点とは，以下のようなものです．

* ステップ入力の目標値はそもそも実現不可能(これを実現しようとすると，加速度$a\to \infty$となるため)
* 急加速を行うことになるため，場合によってはタイヤのスリップと言った影響が無視できなくなる．
* また，急加速に伴い動作開始時に非常に大きな電流が流れるため，設計次第ではモーター回路が焼かれて死ぬ．
* 入力値は最初の一瞬だけ高くて，後は低いという，無駄が多い制御となっている．
* 目標速度を上げると，PIDパラメータの再調整が必要

要は，止まってる物体が突然トップスピードで走り出すことは不可能なので，ステップ入力の目標速度信号が良くないわけです．  
このような問題を克服するため，目標速度を一定値で与えるのではなく，**加速→等速→減速**といった台形加速の目標値信号を与えることを考えます．


ただし，モータの特性上，速度と加速度にはトレードオフの関係があるので，まずは目標とする最高速度に対して実現可能な加速度を見積もっておきます．


---

マウスの直進モデルにおいて，機体の速度$v$と加速度$\dot{v}$の間には次のような関係がありました．

$$\dot{v} = - \frac{K_{E} K_{T} N_{G}^{2} }{R m r^{2}}v + \frac{K_{T} N_{G} }{R m r}V_{m}$$


台形加速による等加速度制御を行うためには，最高速に達するまで同じ加速度を維持できなければいけません．  
直進の制御だけでなく，回転の制御も後で行うことを考えると，コントローラの出力には余裕をもたせたいので，
最高速時，$V_m = 2V_{Bat}\times 0.8$となっているようにします．

よって，目標とする最高速度に達したときに，出すことのできる加速度の最大値$a_{max}$は

$$\dot{v} = \alpha v + \beta Vm$$ とおくと，

$$a_{max} = \alpha v_{Goal} + 1.6 \beta V_{Bat}$$

$$(ただし，\alpha= -\frac{K_{E} K_{T} N_{G}^{2} }{R m r^{2}},\  \beta =\frac{K_{T} N_{G} }{R m r})$$

となるので，目標速度の傾きは$a_{max}$より低くなければならないことがわかりました．  

例として，最高速度を$v_{Goal}=2.0$ [m/s] に設定した場合，

In [6]:
a = mouse.ss.A[1,1]
b = mouse.ss.B[1,0] 
v_bat = 3.6 # 1セル Lipo
v_goal = 2.0 # 最高速度

a_max = a*v_goal + 0.8*b #*2*v_bat
print("a_max: {0:.3f} [m/s^2], v_goal: {1:.3f} [m/s]".format(a_max, v_goal))
print("最高速度に達するまでに {0:.3f}秒".format(v_goal/a_max))

a_max: 6.061 [m/s^2], v_goal: 2.000 [m/s]
最高速度に達するまでに 0.330秒


となるので，$a_{max}=6.061 [m/s^2]$が限界のようです．

このように，加速度と最高速度は独立には設定できないので，先に一方を決めてしまった後に，
もう一方を実現可能な解の中から選ぶようにしたほうが合理的であると言えるでしょう．

そこで，下の例では，目標速度$v_{GOAL}$から算出した限界加速度$a_{max}$に0〜1のゲイン(acc_gain)を掛けることで，
目標軌道を決定しています．

>```python
# 限界加速度を計算
a_max = a*v_goal + 0.8*b 
acc_ref = a_max * acc_gain/100
```

それでは，台形加速軌道に対するフィードバック制御系の設計を行ってみましょう．

In [7]:
a = mouse.ss.A[1,1]
b = mouse.ss.B[1,0] 

# 台形加速軌道を与えたときの，FB制御系のシミュレーション
@interact(Kp=(0.01, 10, 0.01), Ki=(0, 5, 0.001), Kd=(0, 5, 0.01), v_goal=(0,4,0.1), acc_gain=(0,100,0.1))
def sim_closed_loop(Kp=1.0, Ki=0, Kd=0, v_goal=1, acc_gain=100):
    # PID 制御器 の作成

    # 限界加速度を計算
    a_max = a*v_goal + 0.8*b 
    acc_ref = a_max * acc_gain/100
    
    # PIDコントローラ（伝達関数形式）の作成
    # 純粋微分器は計算不可能なので，1次ローパスフィルタを接続した実用微分器(Kd*s/(gamma*Kd*s+1))を使う
    D_COEFFICIENT = 0.1 # 微分係数
    Kp = matlab.tf([Kp], [1])
    Ki = matlab.tf([Ki], [1, 0])
    Kd = matlab.tf([Kd, 0], [D_COEFFICIENT*Kd, 1])  
    K = Kp+Ki+Kd
#     K =matlab.c2d(K, 0.001)

    dmux = StateSpace(0,np.array([0,0,0]),0,np.array([0,1,0]) )
    P = series(mouse.ss , dmux) # 3出力あるうちの2番目(v)だけを取り出す． 

    # 閉ループ系の伝達関数を計算
    Ysys = matlab.feedback(K*P, 1, -1)  # ネガティブフィードバック
    Esys = matlab.feedback(1, K*P, -1)
    Usys = K/(1+K*P)
    
    # 線形シミュレーション
    SIM_TIME = 3    # シミュレーション時間
    SAMPLE_RATE = 1000 # サンプリングレート
    
    t = np.linspace(0, SIM_TIME, SIM_TIME*SAMPLE_RATE)
    ref = acc_ref*t
    ref[ref>v_goal] = v_goal
    yout, T, xout = matlab.lsim(Ysys, ref, t)
    #     Eout, T, xout = matlab.lsim(Esys, ref, t)
    Uout, T, xout = matlab.lsim(Usys, ref, t)

    
    if np.amax(Uout) > 1.0 :
        _is_saturated = True
    else :
        _is_saturated = False
        
    # シミュレーション結果のプロット
    fig, (axL, axR) = plt.subplots(ncols=2, figsize=(16,6))
    
    axL.plot(t, yout)
    axL.plot(t, ref)    
    axL.set_xlim(0, SIM_TIME);  axL.set_ylim(0,5);
    axL.set_xlabel('time t [s]'); axL.set_ylabel('Speed vx [m/s]')
    axL.set_title('Speed Control');  
    # axL.tick_params(labelsize=13)

    if _is_saturated: axR.plot(T, Uout, 'r',  lw=5)
    else: axR.plot(T, Uout, 'b')
    axR.set_xlim(0, SIM_TIME);
    axR.set_xlabel('time t [s]');axR.set_ylabel('duty [.]')
    axR.set_title('Average Duty Output To Two Motors');  
    # axR.tick_params(labelsize=13)

    plt.show();


どうでしょうか？先程よりも調整が楽になったことが分かるでしょうか？

先程のステップ指令値の場合には，目標速度を変更したらPIDパラメータの再調整が必要でしたが，  
こちらの台形加速の場合は一旦PIDのパラメータを決めてしまえば，目標速度が変わってもほぼ同じような制御性能を発揮してくれます．  
また，実現可能な加速度・最高速度の軌道を設定したお陰で，常に出力が80%以内に収まっていることが確認できます．

指定した目標値に追従するように制御することを目的とした制御系のことを**サーボ(Servo)系**と呼ぶのですが，  
このサーボ系の制御問題においては「どのような目標軌道」を与えるかという点も制御性能に関わる非常に重要な要素であることがわかります．

---

ところで，これまで，グラフの波形をみて，なんとなくいい感じの結果が得られそうなPIDパラメータの調整を行うことができました．  
これによってモーターが制御できそうな気持ちになってきたかもしれません．
でもその一方で，皆さんは次のようなことを考えたかもしれません．

* **「これまでの結果はあくまでも理想的なモデルに対する結果であって，現実には全く違う結果になるのではないか？」**
* **「実際の制御システムはサンプリングによる離散的な制御になるはずだが，これらの影響は無視して良いのか？」**
* **「センサーの観測ノイズやシステムの外乱は制御にどのような影響をもたらすのか？」**

このように思った方はスルドイです．   
実際これまで制御系設計に用いてきたモデルは，外乱やモデルの不確かさが存在しない**確定的なシステム**を対象としてきました．   
なので，シミューレーションに用いたモデルと現実の結果は必ずしも一致しません．じゃあ意味がないのかというと，そうとも限りません．

このあたりの話に関しては，小難しい話が並ぶので，ここでは語らないことにします．  
結論から言えば，フィードバックによって多少のモデル誤差や外乱の影響は減衰するので，あまり気にしなくても大丈夫です．  
また，むだ時間に関しては，サンプリング時間を小さめにとっておくようにしたほうが良いでしょう．
マウスの場合1〜10ms周期位を目安にすると良いかと思います．  
ただし，サンプリング時間を短くするほどノイズを拾いやすくなるので，良いセンサを買うか，フィルタリングを頑張る必要があります．
ココらへんの話が気になる人は別ファイルの`FeedBackControl_part-ex.ipynb`にまとめるのでを参照してください．

---


# 2自由度制御系

フィードバック制御では，センサによって観測した情報に基づいてコントローラーの出力を決定します．
コントローラーのゲインを上げれば，即応性が上がりますが，その分センサに入力されたノイズも一緒に増幅してしまいます．
また，サンプリングに伴うむだ時間も存在するため，現実にはコントローラーのゲインを上げすぎるとかえって制御が安定しないということが起こりえます．